In [20]:
import pandas as pd# Used for working with data in table format (like Excel)
from scipy.optimize import linprog# Used to solve optimization problems
import numpy as np# Used for working with numbers and arrays


In [21]:
# Load dataset
# Reads the CSV file and stores the data in a table called 'df'
df = pd.read_csv("Stockist_Data.csv")


In [22]:
# Define metadata columns
metadata_cols = ['Partner_id', 'Geography', 'Stockist_Type', 'Scheme_Type', 'Sales_Value_Last_Period', 
                 'Sales_Quantity_Last_Period', 'MRP', 'Growth_Percentage', 'Discount_Applied', 
                 'Bulk_Purchase_Tendency', 'New_Stockist', 'Feedback_Score']


In [23]:
# Identify product columns (excluding metadata)
product_cols = [col for col in df.columns if col not in metadata_cols]


In [24]:
# Transform data to have 'Product_id' column
# Reshape the data so each row represents a single product for a stockist (partner)
# 'metadata_cols' are columns we want to keep as they are (like Partner ID, etc.)
# 'product_cols' are the product columns that we want to convert into rows
df_melted = df.melt(id_vars=metadata_cols, value_vars=product_cols, 
                     var_name='Product_id', value_name='Has_Product')
# Keep only the rows where the partner actually has the product (Has_Product == 1)
# Drop the 'Has_Product' column since we don't need it anymore
df_melted = df_melted[df_melted['Has_Product'] == 1].drop(columns=['Has_Product']) # Aggregate sales volume per Partner-Product-Scheme combination
# Group the data by Partner ID, Product ID, and Scheme Type
# Then, for each group, sum up the sales value and quantity from the last period
product_schemes = df_melted.groupby(["Partner_id", "Product_id", "Scheme_Type"]).agg({
    "Sales_Value_Last_Period": "sum",
    "Sales_Quantity_Last_Period": "sum"
    # Reset the index to turn the groupby result back into a regular DataFrame
}).reset_index()

In [25]:
# Prepare data for optimization
optimization_data = product_schemes[["Product_id", "Scheme_Type", "Sales_Value_Last_Period"]]

# Function to perform Linear Programming (LP) optimization for selecting the best schemes for each product
def optimize_schemes(product_group):
    # Get the list of unique schemes available for the product
    schemes = product_group["Scheme_Type"].unique()
    num_schemes = len(schemes)# Count how many unique schemes there are

    # If there are no schemes, return three empty (None) values
    if num_schemes == 0:
        return [None, None, None]  # Ensure blank entries

    # If there are 3 or fewer schemes, return them directly
    # Fill the rest with None to ensure we always return 3 values
    if num_schemes <= 3:
        return list(schemes) + [None] * (3 - num_schemes)

    # If there are more than 3 schemes, apply optimization logic to select the best ones
    # Objective: Maximize sales value (we use negative because linprog minimizes by default)
    c = -product_group.groupby("Scheme_Type")["Sales_Value_Last_Period"].sum().values
    # Bounds: Each scheme can either be selected (1) or not (0), or partially in between
    bounds = [(0, 1) for _ in range(num_schemes)]
    # Run the linear programming optimization
    res = linprog(c, bounds=bounds, method='highs', options={"disp": False})
    # (Currently) return blank values as the final optimized schemes are not yet extracted
    return [None, None, None]


In [26]:
# Apply optimization
# Apply the optimization function to each product to get the best 3 schemes
# This runs the 'optimize_schemes' function for each unique 'Product_id'
optimized_schemes = optimization_data.groupby("Product_id").apply(optimize_schemes).reset_index()
# Split the list of 3 schemes (returned by the function) into separate columns: Scheme_1, Scheme_2, Scheme_3
optimized_schemes[["Scheme_1", "Scheme_2", "Scheme_3"]] = pd.DataFrame(optimized_schemes[0].tolist(), index=optimized_schemes.index)
# Drop the original column that contained the list of schemes (now that we've split it into 3 separate columns)
optimized_schemes = optimized_schemes.drop(columns=[0])


In [27]:
# Group Partner_ids per product
partners_per_product = df_melted.groupby("Product_id")["Partner_id"].apply(list).reset_index()

# Merge with optimized schemes
final_optimized_output = partners_per_product.merge(optimized_schemes, on="Product_id", how="left")
final_optimized_output.to_csv("Top_Optimized_Schemes_with_LP.csv", index=False)